In [57]:
# Import necessary libraries
import h2o
from h2o.automl import H2OAutoML

# Initialize the H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,43 mins 40 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month and 18 days
H2O_cluster_name:,H2O_from_python_unknownUser_v0c9sq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,912 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [58]:
# Load your data into a H2OFrame
data = h2o.import_file('/kaggle/input/optiver-trading-at-the-close/train.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [59]:
# Specify the response and predictor columns
response_column = 'target'
predictor_columns = data.columns
predictor_columns.remove("target")

In [ ]:
# Need to detect best model on reduced dataset due to size
## Determine the ratio for your sample
ratio = 0.0001
## Split frame
splits = data.split_frame(ratios = [ratio], seed = 1234)
# Sample is the first split
sample = splits[0]
print("sample size train+val is:",sample.nrows)

# Split the sample into training and validation sets
train_sample, val_sample = sample.split_frame(ratios=[0.8], seed=1)

# Initialize and train the H2OAutoML model
#aml = H2OAutoML(max_models=20, seed=1)
aml = H2OAutoML(exclude_algos=["StackedEnsemble"])
aml.train(x=predictor_columns, y=response_column, training_frame=train_sample)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

# Predict on the validation set
preds = aml.predict(val_sample)

## Convert the predictions back to a pandas dataframe (optional)
#preds_df = h2o.as_list(preds)

sample size train+val is: 525
AutoML progress: |
12:44:08.637: _train param, Dropping bad and constant columns: [row_id]


12:44:10.342: _train param, Dropping bad and constant columns: [row_id]

█
12:44:10.913: _train param, Dropping bad and constant columns: [row_id]

█
12:44:11.397: _train param, Dropping bad and constant columns: [row_id]

█
12:44:13.336: _train param, Dropping bad and constant columns: [row_id]

████
12:44:15.908: _train param, Dropping bad and constant columns: [row_id]
12:44:16.648: _train param, Dropping bad and constant columns: [row_id]

██
12:44:17.558: _train param, Dropping bad and constant columns: [row_id]
12:44:18.511: _train param, Dropping bad and constant columns: [row_id]

██
12:44:19.998: _train param, Dropping bad and constant columns: [row_id]

██
12:44:22.670: _train param, Dropping bad and constant columns: [row_id]

█
12:44:23.401: _train param, Dropping bad and constant columns: [row_id]

████████████████████████████████████████████

In [ ]:
# Extract best model
best_model = aml.leader

# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

In [ ]:
## Load the best model (tested on sample data)
#model = h2o.load_model("/kaggle/working/StackedEnsemble_BestOfFamily_1_AutoML_1_20231011_114524")

In [ ]:
# Split the data into training and validation sets
train, val = data.split_frame(ratios=[0.8], seed=1)

In [ ]:
## Continue training the model on the full data set
#best_model.train(x=predictor_columns, y=response_column, training_frame=train, max_runtime_secs=18000)

In [ ]:
# Train AutoML using only GLM, DeepLearning, and DRF
aml_full = H2OAutoML(max_runtime_secs = 30, exclude_algos = ["StackedEnsemble"])
#aml_full = H2OAutoML(max_models=20, include_algos = ["DRF","GLM"])
aml_full.train(x = predictor_columns, y = response_column, training_frame = train)

In [ ]:
# View the AutoML Leaderboard
lb = aml_full.leaderboard
print(lb.head(rows=lb.nrows))